In [ ]:
from models.transformers import DecoderOnlyTransformer
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import yaml
import gc
import torch
from torchinfo import summary
import os
import json
import shutil
import wandb
import yaml
from data.datasets import CFGDataset, verify_dataloader
from trainers.GPT_trainer import GPT_Trainer
from trainers.utils import create_optimizer, create_scheduler
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

#Read config yaml file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
print(f"Config loaded successfully.")


import os
os.system("export WANDB_DIR=\tmp")
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


Using device: cuda
Config loaded successfully.


0

In [3]:
from data.CFG_parsers import CFGParser
from data.grammars import GRAMMAR_SIMPLE
parser = CFGParser(GRAMMAR_SIMPLE)
parser.is_valid("3213212123")


False

#### Loading Data

In [2]:
train_dataset =  CFGDataset(
    data_file="cfg_sentences_train_cfg3b.npy", 
    batch_size = config["data"]["batch_size"],
    seq_len = config["data"]["seq_len"],
    eos_token = config["data"]["eos_token"],
    sos_token = config["data"]["sos_token"],
    ) 

val_dataset =  CFGDataset(
    data_file="cfg_sentences_val_cfg3b.npy", 
    
    batch_size = config["data"]["batch_size"],
    seq_len = config["data"]["seq_len"],
    eos_token = config["data"]["eos_token"],
    sos_token = config["data"]["sos_token"],
    ) 


train_loader = DataLoader(train_dataset, 
                          batch_size = None, 
                          num_workers=config["data"]["NUM_WORKERS"] if device == "cuda" else 0, 
                          pin_memory=True)

val_loader = DataLoader(val_dataset, 
                        batch_size=None, 
                        num_workers=config["data"]["NUM_WORKERS"] if device == "cuda" else 0,
                        pin_memory=True)

verify_dataloader(train_loader)
print("_"*50)
verify_dataloader(val_loader)
#print("="*50)
#print("Verify Validation DataLoader")
#verify_dataloader(val_loader)

Verifying dataloader...
Number of batches:  40669
Total number of tokens:  1.998977e+09


Example batch shapes (shifted, golden):  torch.Size([96, 512]) torch.Size([96, 512])
__________________________________________________
Verifying dataloader...
Number of batches:  4518
Total number of tokens:  2.221086e+08
Example batch shapes (shifted, golden):  torch.Size([96, 512]) torch.Size([96, 512])


In [3]:
model_config = {}
model_config = config['model'].copy()

model_config.update({
    'num_classes': config["data"]["vocab_size"] +2 ,#include SOS and ESO tolen
    'seq_len': config["data"]["seq_len"],
    })

model = DecoderOnlyTransformer(**model_config)

for batch in train_loader:
    shifted_transcripts, golden_transcripts = batch
    print("Shape of shifted_transcripts : ", shifted_transcripts.shape)
    print("Shape of golden_transcripts  : ", golden_transcripts.shape)
    break

#model_stats = summary(model, input_data=[shifted_transcripts])
#print(model_stats)
print(f"Total parameters in the model: {sum(p.numel() for p in model.parameters()):,}")
#torch._dynamo.config.skip_nnmodule_hook_guards = False
#model = torch.compile(model, mode="default", fullgraph=True)


Shape of shifted_transcripts :  torch.Size([96, 512])
Shape of golden_transcripts  :  torch.Size([96, 512])
Total parameters in the model: 42,532,613


In [4]:
from trainers.utils.create_scheduler import plot_lr_schedule


torch.cuda.empty_cache()
gc.collect()
#wandb.finish()
trainer = GPT_Trainer(
    model = model, 
    config = config,
    config_file = "config.yaml",
    run_name = "full_rope", 
    device = device
)
wandb.watch(trainer.model, log="all")

trainer.optimizer = create_optimizer.create_optimizer(
    model = model, 
    opt_config = config["optimizer"]
)

trainer.scheduler = create_scheduler.create_scheduler(
    optimizer=trainer.optimizer,
    scheduler_config=config['scheduler'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

#plot_lr_schedule(trainer.scheduler, num_epochs=len(train_loader), train_loader=train_loader)

#trainer.load_checkpoint("/ocean/projects/cis250019p/sfragara/lstm/expts/test/checkpoints/checkpoint-best-metric-model.pth")

Using device: cuda


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lucasfragara (teamlsfr) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



🔧 Configuring Optimizer:
├── Type: ADAMW
├── Base LR: 0.0007
├── Weight Decay: 0.1
├── Parameter Groups:
│   ├── Group: self_attn
│   │   ├── LR: 0.0007
│   │   └── Patterns: []
│   ├── Group: ffn
│   │   ├── LR: 0.0007
│   │   └── Patterns: []
│   └── Default Group (unmatched parameters)
└── AdamW Specific:
    ├── Betas: [0.9, 0.98]
    ├── Epsilon: 1e-08
    └── AMSGrad: False


/jet/home/sfragara/.conda/envs/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📈 Configuring Learning Rate Scheduler:
├── Type: COSINE
├── Cosine Annealing Settings:
│   ├── T_max: 40669 steps
│   └── Min LR: 1e-08
└── Warmup: Disabled


In [5]:
trainer.model.train()

#trainer._validate_epoch(val_loader)
trainer.train_epoch(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
)

generating
Using default generation config...
Generating with greedy search...


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


wandb metrics being logged:  {'train/ce_loss_token': 1.3246285346691717, 'train/perplexity_token': 3.7607882022857666, 'val/ce_loss_token': 1.1964159235358238, 'val/perplexity_token': 3.3082385063171387, 'learning_rate': 0.0006997379248477051}

📊 Metrics (Epoch 500):
├── TRAIN:
│   ├── ce_loss_token: 1.3246
│   └── perplexity_token: 3.7608
└── VAL:
    ├── ce_loss_token: 1.1964
    └── perplexity_token: 3.3082
└── TRAINING:
    └── learning_rate: 0.000700


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 1.201375137556802, 'train/perplexity_token': 3.324685573577881, 'val/ce_loss_token': 1.0131168588995934, 'val/perplexity_token': 2.754171848297119, 'learning_rate': 0.0006989541814204076}

📊 Metrics (Epoch 1000):
├── TRAIN:
│   ├── ce_loss_token: 1.2014
│   └── perplexity_token: 3.3247
└── VAL:
    ├── ce_loss_token: 1.0131
    └── perplexity_token: 2.7542
└── TRAINING:
    └── learning_rate: 0.000699


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 1.1279729470501416, 'train/perplexity_token': 3.089387893676758, 'val/ce_loss_token': 0.9538229033350945, 'val/perplexity_token': 2.595613479614258, 'learning_rate': 0.0006976499377227822}

📊 Metrics (Epoch 1500):
├── TRAIN:
│   ├── ce_loss_token: 1.1280
│   └── perplexity_token: 3.0894
└── VAL:
    ├── ce_loss_token: 0.9538
    └── perplexity_token: 2.5956
└── TRAINING:
    └── learning_rate: 0.000698


...

generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.8176024276139799, 'train/perplexity_token': 2.2650628089904785, 'val/ce_loss_token': 0.7754874136298895, 'val/perplexity_token': 2.1716504096984863, 'learning_rate': 0.000439137975821056}

📊 Metrics (Epoch 17000):
├── TRAIN:
│   ├── ce_loss_token: 0.8176
│   └── perplexity_token: 2.2651
└── VAL:
    ├── ce_loss_token: 0.7755
    └── perplexity_token: 2.1717
└── TRAINING:
    └── learning_rate: 0.000439


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.8161478764663035, 'train/perplexity_token': 2.261770486831665, 'val/ce_loss_token': 0.7698382511734962, 'val/perplexity_token': 2.159416913986206, 'learning_rate': 0.0004260022881512363}

📊 Metrics (Epoch 17500):
├── TRAIN:
│   ├── ce_loss_token: 0.8161
│   └── perplexity_token: 2.2618
└── VAL:
    ├── ce_loss_token: 0.7698
    └── perplexity_token: 2.1594
└── TRAINING:
    └── learning_rate: 0.000426


generating
Using default generation config...
Generating with greedy search...
wandb metrics being logged:  {'train/ce_loss_token': 0.8147241976696759, 'train/perplexity_token': 2.2585527896881104, 'val/ce_loss_token': 0.7988059241324663, 'val/perplexity_token': 2.2228851318359375, 'learning_rate': 0.00041275324125568017}

📊 Metrics (Epoch 18000):
├── TRAIN:
│   ├── ce_loss_token: 0.8147
│   └── perplexity_token: 2.2586
└── VAL:
    ├── ce_loss_token: 0.7988
    └── perplexity_token: 2.2229
└── TRAINING:
    └── learning_rate: 0.000413


[Training LM]:  44%|███▉     | 18016/40669 [45:49<58:11,  6.49it/s, acc_step=1/1, ce_loss_token=0.8147, lr=0.000412, perplexity_token=2.2585]

KeyboardInterrupt: 